In [1]:
pip install pandas sacrebleu markdown


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bert-score



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install beautifulsoup4



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
import markdown
from bs4 import BeautifulSoup
import glob
import os
from bert_score import score

In [6]:
# Load the CSV file containing the generated answers
llm_answers = pd.read_csv('/home/subin/Desktop/subin/llm_answers.csv')

# Function to extract text from Markdown files
def extract_text_from_markdown(file_path):
    with open(file_path, 'r') as file:
        md_content = file.read()
    # Convert Markdown to HTML
    html_content = markdown.markdown(md_content)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    # Extract and return text
    return soup.get_text()

# Load and extract text from all Markdown files
markdown_folder = '/home/subin/Desktop/subin/ritsu_bot/markdown_files'
all_reference_texts = {}
for file_path in glob.glob(os.path.join(markdown_folder, '*.md')):
    filename = os.path.basename(file_path)
    all_reference_texts[filename] = extract_text_from_markdown(file_path).strip()

# Extract reference texts and generated answers for BERTScore
references = []
hypotheses = llm_answers['Answer'].str.strip().tolist()

for index, row in llm_answers.iterrows():
    filename = row['Filename']
    if filename in all_reference_texts:
        reference_text = all_reference_texts[filename]
    else:
        print(f"Warning: {filename} not found in markdown folder")
        reference_text = ""  # Assign empty text or handle as needed
    references.append(reference_text)

# Filter out empty references and corresponding hypotheses
filtered_references = [ref for ref in references if ref]
filtered_hypotheses = [hyp for ref, hyp in zip(references, hypotheses) if ref]

# Calculate BERTScore
P, R, F1 = score(filtered_hypotheses, filtered_references, lang="en", verbose=True)

# Average F1 score
average_f1 = F1.mean().item()
print(f"Average BERTScore F1: {average_f1}")

# Add the BERT F1 scores to the DataFrame
llm_answers['BERTScore F1'] = F1.tolist() + [None] * (len(hypotheses) - len(F1))

# Save the results to a new CSV file
llm_answers.to_csv('/home/subin/Desktop/subin/BERTScore.csv', index=False)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 8.34 seconds, 23.38 sentences/sec
Average BERTScore F1: 0.8504440188407898
